In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder

In [21]:
import pickle

In [22]:
## Load Data
df = pd.read_csv('Churn_Modelling.csv')


In [23]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [24]:
## Remove unecessary Features
df = df.drop(['RowNumber','CustomerId','Surname'], axis=1)
df.head()


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## Encoding categorical features

In [25]:
## 'Gender'
label_encoder = LabelEncoder()
df['Gender'] = label_encoder.fit_transform(df['Gender'])
df['Gender'].unique()

array([0, 1])

In [26]:
## Geography
one_hotencoder = OneHotEncoder()
geo_encoder = one_hotencoder.fit_transform(df[['Geography']]).toarray()

In [27]:
geo_encoder

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [28]:
one_hotencoder.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [29]:
geo_encoded_df=pd.DataFrame(geo_encoder,columns=one_hotencoder.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [30]:
## Combine one hot encoder columns with the original data
df=pd.concat([df.drop('Geography',axis=1),geo_encoded_df],axis=1)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [31]:
## Save the encoders and sscaler
with open('artifacts/label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder,file)

with open('artifacts/onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(one_hotencoder,file)


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CreditScore        10000 non-null  int64  
 1   Gender             10000 non-null  int32  
 2   Age                10000 non-null  int64  
 3   Tenure             10000 non-null  int64  
 4   Balance            10000 non-null  float64
 5   NumOfProducts      10000 non-null  int64  
 6   HasCrCard          10000 non-null  int64  
 7   IsActiveMember     10000 non-null  int64  
 8   EstimatedSalary    10000 non-null  float64
 9   Exited             10000 non-null  int64  
 10  Geography_France   10000 non-null  float64
 11  Geography_Germany  10000 non-null  float64
 12  Geography_Spain    10000 non-null  float64
dtypes: float64(5), int32(1), int64(7)
memory usage: 976.7 KB


In [33]:
data = df

In [34]:
## DiVide the dataset into indepent and dependent features
X=data.drop('Exited',axis=1)
y=data['Exited']

## Split the data in training and tetsing sets
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

## Scale these features
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)


In [35]:
X_train


array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [36]:
with open('artifacts/scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

# ANN Training

Parameters:
1) Sequence NEtwork
2) Dense Layer
3) Activation Function
4) Optimizer -> Backpropogation
5) Loss Function
6) Metrics -> Performance Evaluation

Steps: 
Training -> Logs -> Folder -> Tensorboard -> Visualization

A Dense layer is a fully connected layer where every neuron is connected to every neuron in the previous layer.

In [39]:
# Input is in the form X = [x1, x2, x3, ..., x10]
(X_train.shape[1],)

(12,)

In [38]:
## Build ANN model

import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [ ]:
## Create model Structure == input ->two hidden layer only -> output
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),  ##HL1 connected to input layer befor this layer
    Dense(32, activation='relu'),   ##HL2
    Dense(1, activation='sigmoid')   ##Output layer
]
)

In [41]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [42]:
# Model Training on the data using Adam Optimizer
## Set learning parameters
import tensorflow

opt = tensorflow.keras.optimizers.Adam(learning_rate=0.1)
loss = tensorflow.keras.losses.BinaryCrossentropy()

In [ ]:
## Compile model
model.compile(optimizer=opt, loss = loss, metrics=['accuracy'])


In [44]:
## Set up Tensorboard -> TensorBoard is a dashboard that visually shows what’s happening inside your neural network while it is training
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [45]:
## Set up Early Stopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)


In [46]:
### Train the model
history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100


250/250 [==============================] - 4s 8ms/step - loss: 0.4843 - accuracy: 0.7920 - val_loss: 0.4112 - val_accuracy: 0.8035
Epoch 2/100
250/250 [==============================] - 1s 5ms/step - loss: 0.4547 - accuracy: 0.7935 - val_loss: 0.4440 - val_accuracy: 0.8035
Epoch 3/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4772 - accuracy: 0.7977 - val_loss: 0.4923 - val_accuracy: 0.8105
Epoch 4/100
250/250 [==============================] - 1s 4ms/step - loss: 0.5143 - accuracy: 0.7975 - val_loss: 0.4984 - val_accuracy: 0.8035
Epoch 5/100
250/250 [==============================] - 2s 6ms/step - loss: 0.5087 - accuracy: 0.7945 - val_loss: 0.4991 - val_accuracy: 0.8035
Epoch 6/100
250/250 [==============================] - 1s 5ms/step - loss: 0.5091 - accuracy: 0.7945 - val_loss: 0.4991 - val_accuracy: 0.8035
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.5088 - accuracy: 0.7945 - val_loss: 0.4958 - val_accuracy: 0.80

When you save a model as .h5, it includes:

🧠 Model architecture (layers, connections)

⚖️ Weights & biases

⚙️ Optimizer state (Adam, learning rate, etc.)

📉 Loss & metrics config

In [47]:
model.save('model.h5')

c:\Users\Sandesh Chipkar\Desktop\Churn_prediction_ANN\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [48]:
# Load Tesnor Board extension
%load_ext tensorboard

In [54]:
%tensorboard --logdir logs/fit


Reusing TensorBoard on port 6006 (pid 25608), started 0:05:54 ago. (Use '!kill 25608' to kill it.)